In [ ]:
import MySQLdb
import pandas as pd
import db2048
import traceback
import json
import logging

In [13]:
db = MySQLdb.connect("localhost", "u2048", "u2048passord", init_command="use testDB")

In [14]:
db.encoding = "utf-8"


In [15]:
#!sudo pip3 install flask



In [16]:
def transform_rating(df):
    n = len(df)
    data = df.to_dict()
    rating = []
    for i in range(n):
        rating.append({
            "place": i + 1,
            "name": data["NAME"][i],
            "score": data["SCORE"][i]
        })
    return rating

In [ ]:
import flask

app = flask.Flask(__name__)

@app.route('/rating')
def rating():
    limit = int(flask.request.args.get('limit', 5))
    if bool(flask.request.args.get('json', "")):
        return json.dumps(transform_rating(db2048.get_rating(db, limit=limit)))
    else:
        return db2048.get_rating(db, limit=limit).to_html(index=False)

@app.route('/add_user')
def add_user():
    user_name = "no name :("
    escaped_name = "no name :("
    try:
        user_name = str(flask.request.args['name'])
    except Exception:
        return traceback.format_exc(), 400
    try:
        escaped_name = db.escape_string(user_name)
        user_id, new = db2048.add_user(db, name=escaped_name)
        return json.dumps({"success": True, "user_id": user_id, "new": new}), 200
    except Exception:
        logging.exception("Can't add user {}, {}: ".format(repr(user_name), repr(escaped_name)))
        return traceback.format_exc(), 500
    
@app.route('/get_current_version')
def get_current_version():
    try:
        user_id = int(flask.request.args['user_id'])
    except Exception:
        return traceback.format_exc(), 400
    try:
        version = db2048.get_current_version(db, user_id=user_id)
        return json.dumps({"success": True, "version": version}), 200
    except Exception:
        return traceback.format_exc(), 500
    
@app.route('/add_play')
def add_play():
    try:
        user_id = int(flask.request.args['user_id'])
        score = float(flask.request.args['score'])
        version = int(flask.request.args['version'])
    except Exception:
        return traceback.format_exc(), 400
    try:
        play_id, success = db2048.add_play(db, user_id=user_id, score=score, version=version)
        return json.dumps({"success": success, "play_id": play_id}), 200
    except Exception:
        return traceback.format_exc(), 500


@app.route('/')
def index():
    return("Hello World!")


app.run(port=8890, host="0.0.0.0")

In [ ]:
import flask


In [ ]:
help(app.run)


[{'place': 1, 'name': 'Happy Year', 'score': 81.0},
 {'place': 2, 'name': 'Testing Vasya', 'score': 64.0},
 {'place': 3, 'name': 'Testing Vasya', 'score': 32.0},
 {'place': 4, 'name': 'Happy Year', 'score': 27.0},
 {'place': 5, 'name': 'Testing Vasya', 'score': 16.0}]

In [4]:
len(db2048.get_rating(db, limit=5))

5

In [ ]:
!jupyter nbconvert server.ipynb --help


In [ ]:
!ls